In [48]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [49]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import model_selection

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn import datasets

from keras.datasets import mnist

from keras.models import Sequential, load_model

from keras.layers import Dense, Dropout, Flatten

from keras.layers.convolutional import Conv2D, MaxPooling2D

from keras.utils.np_utils import to_categorical

import cv2
import os
import glob

%matplotlib inline

In [50]:
img_dir = "../input/chest-xray-pneumonia/chest_xray/train/NORMAL" # Enter Directory of all images 
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
X=[]
y=[]
for f1 in files:
    img = cv2.imread(f1)
    img = cv2.resize(img, (100,100))
    X.append(np.array(img))
    y.append(0)
n_NORMAL = len(X)

In [51]:
img_dir = "../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA" # Enter Directory of all images 
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
for f1 in files:
    img = cv2.imread(f1)
    img = cv2.resize(img, (100,100))
    X.append(np.array(img))
    y.append(1)
n_PNEUMONIA = len(X)-n_NORMAL

In [52]:
X = np.array(X)
y = np.array(y)

In [53]:
X = X / 255
print(X[0][0])

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [55]:
# Simple neural network
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(100, 100, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1))

# Compilign model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [56]:
model.summary()

In [57]:
# training the model
train = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200, verbose=1)

In [58]:
# Testing the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))

In [59]:
print(train.history['accuracy'])

In [60]:
print(train.history['val_accuracy'])

In [61]:
def plot_scores(train) :
    accuracy = train.history['accuracy']
    val_accuracy = train.history['val_accuracy']
    epochs = range(len(accuracy))
    plt.plot(epochs, accuracy, 'b', label='learning accuracy')
    plt.plot(epochs, val_accuracy, 'r', label='validation accuracy')
    plt.title('Scores')
    plt.legend()
    plt.show()

In [62]:
plot_scores(train)

In [63]:
# Prediction
y_cnn = model.predict_classes(X_test)

In [64]:
cm = confusion_matrix(y_cnn,y_test)
print(cm)
plt.figure(figsize = (12,10))

In [65]:
plt.figure(figsize=(15,25))
n_test = X_test.shape[0]
i=1
for j in range(len(X_test)) :
    if (y_cnn[j] != y_test[j]) & (i<50):
        plt.subplot(10,5,i)
        plt.axis('off')
        plt.imshow(X_test[j])
        pred_classe = y_cnn[j].argmax(axis=-1)
        plt.title('%d / %d' % (y_cnn[j], y_test[j]))
        i+=1

In [66]:
# Deeper CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(100, 100, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(20, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(20, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [67]:
model.summary()

In [68]:
# Training the model
train = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=200, verbose=1)

# Testing the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))

In [69]:
plot_scores(train)

In [70]:
from keras.applications import VGG16

In [71]:
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(100,100,3))
vgg16.trainable = False


In [72]:
vgg16.summary()

In [73]:
model = Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1))

In [74]:
model.summary()

In [75]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [76]:
train = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200, verbose=1)

In [77]:
# Testing the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))

In [78]:
for layer in vgg16.layers[15:]:
    layer.trainable=True
for layer in vgg16.layers[0:15]:
    layer.trainable=False

In [79]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
train = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200, verbose=True)

In [80]:
def plot_scores(train) :
    accuracy = train.history['accuracy']
    val_accuracy = train.history['val_accuracy']
    epochs = range(len(accuracy))
    plt.plot(epochs, accuracy, 'b', label='learning accuracy')
    plt.plot(epochs, val_accuracy, 'r', label='validation accuracy')
    plt.title('Scores')
    plt.legend()
    plt.show()

In [81]:
plot_scores(train)

In [82]:
y_cnn = model.predict_classes(X_test)

In [83]:
plt.figure(figsize=(15,25))
n_test = X_test.shape[0]
i=1
for j in range(len(X_test)) :
    if (y_cnn[j] != y_test[j]) & (i<50):
        plt.subplot(10,5,i)
        plt.axis('off')
        plt.imshow(X_test[j])
        pred_classe = y_cnn[j].argmax(axis=-1)
        plt.title('%d / %d' % (y_cnn[j], y_test[j]))
        i+=1